<a href="https://colab.research.google.com/github/anjaleeDS/Sandbox/blob/master/Rose_WEBscrapingNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import lxml.etree as xml
import requests  # go out and fetch itnative python
import pandas as pd
import numpy as np
import re
import time
import requests


In [3]:
from google.colab import drive
drive.mount('drive')


Mounted at drive


# Pandas way to scrape and clean UC Davis rose catalog 

In [3]:
url = "https://fps.ucdavis.edu/roses/collection.cfm"
requests.get(url)

<Response [200]>

## Get the table data. No details

In [4]:
r=requests.get(url)
df_list= pd.read_html(r.text)

make into a dataframe

In [5]:
df = pd.DataFrame(data=df_list[1],copy=True)
df.shape

(820, 6)

clean the headers

In [6]:
df.columns = df.columns.str.strip() # blanks
df.columns = df.columns.str.replace(r"[^a-zA-Z\d\_]+", "")

df.columns


Index(['Unnamed0', 'Cultivar', 'FlowerColor', 'FlowerColor1', 'PlantType',
       'Fragrance'],
      dtype='object')

take out columns that have no data


In [7]:
for col in df.columns:
  if df[col].count()==0:
    # print(col)
    df = df.drop(col, axis=1)  

df.columns


Index(['Cultivar', 'FlowerColor1', 'PlantType', 'Fragrance'], dtype='object')

### get the detail page URLs

In [8]:
d_url = (re.findall('cultivar.cfm\?varietyid=\d+', r.text)) # get all the onclicks
d_url = pd.DataFrame(data=d_url) # stick them in a df

df = (pd.concat([df,d_url],axis=1)) # concat the dfs together
df = df.rename({df.columns[df.shape[1]-1] :'details'},axis=1) # rename that concated column to the correct column name

# make all of the details into urls with detail
df['details'] = df.apply(lambda x:"https://fps.ucdavis.edu/roses/"+  x['details'], axis=1)


In [9]:
df

,Cultivar,FlowerColor1,PlantType,Fragrance,details
0,A Shropshire Lad ®,light pink,Shrub,moderate,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
1,Abbaye De Cluny ™,apricot & apricot blend,Hybrid Tea & Climbing Hybrid Tea,slight,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
2,About Face ™,orange & orange blend,Grandiflora & Climbing Grandiflora,none,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
3,Above All ™,orange & orange blend,Large-Flowered Climber,moderate - fruity,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
4,Above and Beyond ™,apricot & apricot blend,Large-Flowered Climber,moderate - spice,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
...,...,...,...,...,...
815,Young Lycidas,deep pink,Shrub,intense - delicious tea and old rose with hint...,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
816,You're the One ™,pink blend,Miniature & Climbing Miniature,slight,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
817,Yuletide,medium red,Hybrid Tea & Climbing Hybrid Tea,slight,https://fps.ucdavis.edu/roses/cultivar.cfm?var...
818,Yves Piaget ®,medium pink,Hybrid Tea & Climbing Hybrid Tea,intense,https://fps.ucdavis.edu/roses/cultivar.cfm?var...


## Fetch details from each detail page

### Get detail column names

In [10]:
def get_details(detail_url):
  d = requests.get(detail_url) 
  dfile = (pd.read_html(d.text))
  return dfile[2]

In [11]:
def add_record(detail_df, new_record):
  # detail_df_len = len(detail_df)
  # detail_df.loc[detail_df_len] = new_record
  # df = detail_list.merge(df, how='outer')
  detail_df = detail_df.merge(new_record, how='outer')
  return detail_df


In [12]:
#only take the header (col1) from first detail page grab
data_dump = get_details(df['details'][0])  

#create the detail df with columns as fields and add first record
big_cols = list(data_dump[:][0])
detail_df = pd.DataFrame(columns=big_cols)

#first record
detail_df_len = len(detail_df)
detail_df.loc[detail_df_len] = list(data_dump[:][1])
detail_df

,Cultivar Name:,Synonyms:,Patent:,Description:,Registration Name:,Classification:,Climber:,Petal Color:,Hybridizer:,Flower Form:,Fragrance:,Foliage:,Prickle:,Awards:,References:,Parentage:
0,A Shropshire Lad ®,'L/58/88',David Austin Roses Limited PP10607,Introduction Year: 1996; sub-type: English Lea...,Ausled,S - Shrub,No,light pink,"Austin, David","flowers strikingly beautiful soft peachy pink,...",moderate,"medium size, medium green, semi-glossy",thornless or almost thornless,RHS Award of Garden Merit (2012),Modern Roses 12 www.davidaustinroses.com www...,Heritage x unnamed seedling


### append rose records onto ddf using column name as join on field

In [13]:
i=1
while i in range(df.shape[0]):
  # next record from main df
  detail_dump = get_details(df['details'][i])
  # next_record_df = prep_record(detail_dump)

  # transpose it
  t_dd = detail_dump.transpose()
  # copy first row as column names and recopy df to start from df[1]
  t_dd.columns= t_dd.iloc[0]
  t_dd = t_dd[1:]


  # merge it with the rest of the detail_df
  detail_df = detail_df.merge(t_dd, how='outer')
  # print("hi", t_dd, detail_df)
  i= i+1


In [14]:
detail_df

,Cultivar Name:,Synonyms:,Patent:,Description:,Registration Name:,Classification:,Climber:,Petal Color:,Hybridizer:,Flower Form:,Fragrance:,Foliage:,Prickle:,Awards:,References:,Parentage:,Ploidy:
0,A Shropshire Lad ®,'L/58/88',David Austin Roses Limited PP10607,Introduction Year: 1996; sub-type: English Lea...,Ausled,S - Shrub,No,light pink,"Austin, David","flowers strikingly beautiful soft peachy pink,...",moderate,"medium size, medium green, semi-glossy",thornless or almost thornless,RHS Award of Garden Merit (2012),Modern Roses 12 www.davidaustinroses.com www...,Heritage x unnamed seedling,NaN
1,Abbaye De Cluny ™,'Romantic Seranade',The Conard-Pyle Company PP09609,NaN,Meibrinpay,HT & Cl HT - Hybrid Tea & Climbing Hybrid Tea,No,apricot & apricot blend,Meilland International SA,"orange apricot, 4¼ in., full, borne mostly singly",slight,"medium size, dark green, semi-glossy",moderate,"Gold Medal, Belfast, 1995",Modern Roses 12,Just Joey × (MEIrestif × MEInarual),NaN
2,About Face ™,'Pride of Cheshire',"Weeks Wholesale Rose Grower, Inc. PP17305",NaN,Wekosupalz,Gr & Cl Gr - Grandiflora & Climbing Grandiflora,No,orange & orange blend,"Carruth, Tom","deep orange gold, reverse burnished red, 9-12 ...",none,"large, dark green, semi-glossy","moderate, average, almost straight, brown","AARS, USA, 2005 Portland's Best Rose, Portland...",Modern Roses 12 www.helpmefind.com United St...,[(O Sole Mio x seedling) x Midas Touch] x Hot ...,NaN
3,Above All ™,NaN,NaN,"Intro Year: 2015; fine gardening, 2015 article...",Chewesic,LCl - Large-Flowered Climber,Yes,orange & orange blend,"Warner, Chris","bud form, pointed and ovoid; flowers dark/salm...",moderate fruity,"medium-size, medium green, semi-glossy","almost straight, moderate, light brown",NaN,Modern Roses 12 www.helpmefind.com United St...,Westerland × Sweet Magic,NaN
4,Above and Beyond ™,'First Editions® Above and Beyond™ Rose',PP24463,First Editions® collection. Registered in 2014...,Zleeltonstrack,LCl - Large-Flowered Climber,Yes,apricot & apricot blend,"Zlesak, David","buds medium, pointed, ovoid; flowers light apr...",moderate spice,"light - medium green, semi-glossy; fall foliag...","straight to slightly hooked, moderate",Earth-Kind Rose,Modern Roses 12 www.helpmefind.com United St...,(Lemon Fluff (Curlem)) × (Rosa virginiana x Ro...,tetraploid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,Young Lycidas,NaN,David Austin Roses Ltd. PP20960,English Old Rose Hybrid introduced in 2008. Ac...,Ausvibrant,S - Shrub,No,deep pink,"David Austin Roses, Ltd.","buds, medium; flowers deep pink with lavender ...",intense delicious tea and old rose with hints ...,"medium-size, medium green, semi-glossy","deeply concave, few, red-brown","Perfume Prize, Barcelona Rose Trials, Spain, 2...",Modern Roses 12 www.davidaustinroses.com www...,"Unnamed, unpatented seedling (pink English-typ...",NaN
816,You're the One ™,NaN,NaN,Introduced in 2012,Wektaclagoma,Min & Cl Min - Miniature & Climbing Miniature,No,pink blend,"Carruth, Tom","flowers white edged pink blushing darker, reve...",slight,"medium-siz, dark green, glossy","almost straight, moderate, beige","Award of Excellence, American Rose Society, US...",Modern Roses 12 www.helpmefind.com www.weeks...,Santa Claus × Goldmarie,NaN
817,Yuletide,NaN,Joseph H. Hill Company PP01391,NaN,Yuletide,HT & Cl HT - Hybrid Tea & Climbing Hybrid Tea,No,medium red,"Joseph H. Hill, Co.","bud ovoid; flowers currant-red, 3½-4½ in., 43 ...",slight,"leathery, semi-glossy",NaN,NaN,Modern Roses 12,Hill Crest × Silver Kenny,NaN
818,Yves Piaget ®,"'Queen Adelaide', 'The Royal Brompton Rose'",The Conard-Pyle Company PP06895,NaN,Meivildo,HT & Cl HT - Hybrid Tea & Climbing Hybrid Tea,No,medium pink,"Meilland, Mrs. Marie-Louise","bud globular, fairly large; flowers Neyron pin...",intense,"medium size, dark green, semi-glossy",NaN,"Golden Rose & Fragrance Award, Geneve, 1982 Go...",Modern Roses 12,((Pharaoh × Peace) × (Chrysler Imperial × Char...,NaN


### Write to file

In [15]:
def make_filename(prefix):
  dateTimeObj = pd.to_datetime('today')
  dateStr = dateTimeObj.strftime("%m_%d_%Y")
  fname = (prefix + dateStr + ".csv")
  return (fname)

In [17]:
%cd drive/My Drive/Colab Notebooks/Sandbox/Roses/UCDavis
%ls

/content/drive/My Drive/Colab Notebooks/Sandbox/Roses/UCDavis


In [18]:
fileprefix = "UCDavisRoses"
fn = make_filename(fileprefix)
detail_df.to_csv(fn)
!cp {fn} .
%ls

cp: 'UCDavisRoses07_05_2021.csv' and './UCDavisRoses07_05_2021.csv' are the same file
UCDavisRoses07_05_2021.csv


# Get rose list from helpmefind

## HMF Page Structures **Alpha lists**
### hmf alpha URL structure:
Main rose search by name page: 
`https://www.helpmefind.com/rose/plants.php`

Alpha List main page, no letter selected:
`[Main rose search]?tab=2`

Alpha List, with letter selected: <letter> is A to Z <page number> is 0,1-9, a-f, 10-19, 1a-1f, 20-29, 2a-2f, 30-39. 0 is the first page of the letter
`[Main rose search]?grp=<*letter*>&t=2&qn=<*page number*>&qc=0`


Alpha List, with letter selected, after first page: <page number> is 1-9, a-f, 10-19, 1a-1f, 20-29, 2a-2f, 30-39
`[Alpha List, first letter page]&qn=<*page number*>`

test the hmf alphabetical list page for A

In [7]:
hmf = "https://www.helpmefind.com/rose/plants.php?grp=A&t=2"
hmftest ="https://www.helpmefind.com/rose/plants.php?grp=A&t=2&qn=9f&qc=0"
requests.get(hmftest)

<Response [200]>

In [ ]:
r=requests.get(hmf)
df_list= pd.read_html(r.text)

In [ ]:
df = pd.DataFrame(data=df_list[1],copy=True)
df.shape